In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import AllChem

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import time
from scipy.stats import spearmanr
from numpy.random import seed
import seaborn as sns
import scipy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pickle
import os
# Initialize model and move to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = pd.read_csv("../data/Tg.csv")

molecules = df.Smiles.apply(Chem.MolFromSmiles)
fp = molecules.apply(lambda m: AllChem.GetMorganFingerprint(m, radius=3))
fp_n = fp.apply(lambda m: m.GetNonzeroElements())
HashCode = []
for i in fp_n:
    for j in i.keys():
        HashCode.append(j)
                
unique_set = set(HashCode)
unique_list = list(unique_set)
Corr_df = pd.DataFrame(unique_list).reset_index()
                
MY_finger = []
for polymer in fp_n:
    my_finger = [0] * len(unique_list)
    for key in polymer.keys():
        index = Corr_df[Corr_df[0] == key]['index'].values[0]
        my_finger[index] = polymer[key]
    MY_finger.append(my_finger)
X = pd.DataFrame(MY_finger)

# filter input into the most popular X substructures
Zero_Sum = (X == 0).astype(int).sum()
NumberOfZero = 6862
print(len(Zero_Sum[Zero_Sum < NumberOfZero]))

Columns = Zero_Sum[Zero_Sum < NumberOfZero].index
Substructure_list = list(polymer.keys())
X_count = X[Columns]

Y = df['Tg'].values

In [ ]:
pickle_out = open("Corr_All.pickle","wb")
pickle.dump(Corr_df, pickle_out)
pickle_out.close()

pickle_out = open("unique_list_All.pickle","wb")
pickle.dump(unique_list, pickle_out)
pickle_out.close()

pickle_out = open("polymer.keys_All.pickle","wb")
pickle.dump(Substructure_list, pickle_out)
pickle_out.close()

pickle_out = open("Columns_All.pickle","wb")
pickle.dump(Columns, pickle_out)
pickle_out.close()

In [ ]:
xtrain, xtemp, ytrain, ytemp = train_test_split(X_count, Y, test_size=0.2, random_state=11)
xval, xtest, yval, ytest = train_test_split(xtemp, ytemp, test_size=0.5, random_state=42)

bs = 128

# Assuming X and Y are your feature and target datasets respectively
xtrain_tensor = torch.tensor(xtrain.values).float()
ytrain_tensor = torch.tensor(ytrain).float()
xval_tensor = torch.tensor(xval.values).float()
yval_tensor = torch.tensor(yval).float()
xtest_tensor = torch.tensor(xtest.values).float()
ytest_tensor = torch.tensor(ytest).float()

# Create DataLoader for batch processing
train_data = TensorDataset(xtrain_tensor, ytrain_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=bs, shuffle=True)
val_data = TensorDataset(xval_tensor, yval_tensor)
val_loader = DataLoader(dataset=val_data, batch_size=bs)
test_data = TensorDataset(xtest_tensor, ytest_tensor)
test_loader = DataLoader(dataset=test_data, batch_size=bs)

In [ ]:
n_1 = 512
n_2 = 512
n_3 = 64

class BayesianLinear(nn.Module):
    # Bayesian Linear layer
    def __init__(self, in_features, out_features):
        super(BayesianLinear, self).__init__()
        # parameters for the mean
        self.mean = nn.Parameter(torch.Tensor(out_features, in_features))
        # parameters for the standard deviation
        self.log_std = nn.Parameter(torch.Tensor(out_features, in_features))
        self.reset_parameters()

    def reset_parameters(self):
        # Initialize the parameters
        nn.init.normal_(self.mean, 0, 0.1)
        nn.init.normal_(self.log_std, -2.1, 0.7)

    def forward(self, x):
        # Sample weights
        std = torch.exp(self.log_std)
        eps = torch.randn_like(std)
        weights = self.mean + eps * std
        return F.linear(x, weights)

class BayesianNeuralNetwork(nn.Module):
    def __init__(self, n_features):
        super(BayesianNeuralNetwork, self).__init__()
        self.bayesian1 = BayesianLinear(n_features, n_1)
        self.bayesian2 = BayesianLinear(n_1, n_2)
        self.bayesian3 = BayesianLinear(n_2, n_3)
        self.bayesian4 = BayesianLinear(n_3, 2)

    def forward(self, x):
        x = F.relu(self.bayesian1(x))
        x = F.relu(self.bayesian2(x))
        x = F.relu(self.bayesian3(x))
        output = self.bayesian4(x)
        mean = output[:, 0]
        # Ensure standard deviation is positive
        std = torch.exp(output[:, 1].clamp(-10, 5))
        # std = torch.exp(output[:, 1])
        return torch.distributions.Normal(mean, std)

def combined_loss(targets, mean, std, w1, w2):
    # Mean Absolute Error term
    mae = torch.abs(targets - mean).mean()

    # Negative Log-Likelihood term
    variance = std**2
    nll_first_term = torch.log(2 * torch.pi * variance) / 2
    nll_second_term = ((targets - mean)**2) / (2 * variance)
    nll = torch.mean(nll_first_term + nll_second_term)

    # Combined loss
    return w1 * mae + w2 * nll

In [ ]:
def train_model(model, train_loader, w1, w2, epochs):
    optimizer = torch.optim.Adam(model.parameters())
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            distribution = model(data)
            mean, std = distribution.mean, distribution.stddev
            loss = combined_loss(target, mean, std, w1, w2)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch}, Loss: {total_loss / len(train_loader)}')

model_BNN= BayesianNeuralNetwork(n_features=xtrain.shape[1]).to(device)

w1 = 0.0
w2 = 1
# Train the model
train_model(model_BNN, train_loader, w1, w2, epochs=1000)

In [ ]:
def predict_with_uncertainty(model, input_data, n_samples=500):
    model.eval()
    all_samples = torch.zeros((n_samples, input_data.size(0)))
    with torch.no_grad():
        for i in range(n_samples):
            distribution = model(input_data)
            all_samples[i] = distribution.sample()  # Sample from the distribution
    mean = all_samples.mean(0)
    std = all_samples.std(0)
    return mean, std

# Convert test data to tensor and move to GPU
xtest_tensor = torch.tensor(xtest.values, dtype=torch.float32).to(device)

# Predictions
mean_train, std_train = predict_with_uncertainty(model_BNN, xtrain_tensor.to(device))
mean_test, std_test = predict_with_uncertainty(model_BNN, xtest_tensor)

In [ ]:
mean_train = mean_train.numpy()
std_train = std_train.numpy()
mean_test = mean_test.numpy()
std_test = std_test.numpy()

In [ ]:
# Calculate absolute errors and Spearman's Rank Correlation Coefficient for the training set
abs_error_train = abs(ytrain - mean_train)
spearman_corr_train, p_value_train = spearmanr(abs_error_train, std_train)

# Calculate absolute errors and Spearman's Rank Correlation Coefficient for the test set
abs_error_test = abs(ytest - mean_test)
spearman_corr_test, p_value_test = spearmanr(abs_error_test, std_test)

# Organize the results in a dictionary
spearman_results = {
    'Spearman_Correlation': [spearman_corr_train, spearman_corr_test],
    'p_value': [p_value_train, p_value_test]
}

# Convert the dictionary to a DataFrame with 'Train' and 'Test' as index
spearman_df = pd.DataFrame(spearman_results, index=['Train', 'Test'])

# Display the DataFrame
spearman_df

In [ ]:
# Create the plot
plt.figure(figsize=(5, 5))
plt.scatter(abs_error_train, std_train, alpha=0.5)

# Add labels and title
plt.xlabel('Absolute Error (train)', fontsize=14)
plt.ylabel('Standard Deviation (train)', fontsize=14)
plt.title('Absolute Error vs Standard Deviation (train Set)', fontsize=16)

# Optionally, add grid for better readability
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Create the plot
plt.figure(figsize=(5, 5))
plt.scatter(abs_error_test, std_test, alpha=0.5)

# Add labels and title
plt.xlabel('Absolute Error (Test)', fontsize=14)
plt.ylabel('Standard Deviation (Test)', fontsize=14)
plt.title('Absolute Error vs Standard Deviation (test Set)', fontsize=16)

# Optionally, add grid for better readability
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Ensure that ytest, mean_predictions, and std_dev_predictions are 1D arrays
ytrain_1d = np.ravel(ytrain)
ytest_1d = np.ravel(ytest)
mean_train_1d = np.ravel(mean_train)
std_train_1d = np.ravel(std_train)
mean_test_1d = np.ravel(mean_test)
std_test_1d = np.ravel(std_test)

# Metric calculation
mae_train = mean_absolute_error(ytrain_1d, mean_train_1d)
rmse_train = np.sqrt(mean_squared_error(ytrain_1d, mean_train_1d))
r2_train = r2_score(ytrain_1d, mean_train_1d)

mae_test = mean_absolute_error(ytest_1d, mean_test_1d)
rmse_test = np.sqrt(mean_squared_error(ytest_1d, mean_test_1d))
r2_test = r2_score(ytest_1d, mean_test_1d)

# Organize the metrics into a dictionary with three keys for the three metrics
metrics = {
    'MAE': [mae_train, mae_test],
    'RMSE': [rmse_train, rmse_test],
    'R2': [r2_train, r2_test]
}

# Convert the dictionary to a DataFrame
metrics_df = pd.DataFrame(metrics, index=['Train', 'Test'])

# Display the DataFrame
metrics_df

In [ ]:
# Set up the matplotlib figure with two subplots: one for train and one for test
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11, 5))

# Plotting for the training set on the left subplot
axes[0].errorbar(ytrain_1d, mean_train_1d, 
                 yerr=std_train_1d, 
                 fmt='o', ecolor='lightgray', mec='blue', mfc='skyblue', 
                 alpha=0.7, capsize=5, label='Train Prediction')

# Plot a line for perfect predictions for reference
axes[0].plot(ytrain_1d, ytrain_1d, 'r--', label='Perfect predictions')

# Define the ticks for the x and y axes
axes[0].set_xticks(np.arange(-100, 401, 100))
axes[0].set_yticks(np.arange(-100, 401, 100))

# Add labels and title
axes[0].set_xlabel('Actual Values', fontsize=14)
axes[0].set_ylabel('Predicted Values', fontsize=14)
axes[0].set_title('Training Set Predictions - Tg', fontsize=14)

# Add legend
axes[0].legend(fontsize=14)

# Plotting for the test set on the right subplot
axes[1].errorbar(ytest_1d, mean_test_1d, 
                 yerr=std_test_1d, 
                 fmt='o', ecolor='lightblue', mec='green', mfc='lightgreen', 
                 alpha=0.7, capsize=5, label='Test Prediction')

# Plot a line for perfect predictions for reference
axes[1].plot(ytest_1d, ytest_1d, 'r--', label='Perfect predictions')

# Define the ticks for the x and y axes
axes[1].set_xticks(np.arange(-100, 401, 100))
axes[1].set_yticks(np.arange(-100, 401, 100))

# Add labels and title
axes[1].set_xlabel('Actual Values', fontsize=14)
axes[1].set_title('Test Set Predictions - Tg', fontsize=14)

# Add legend
axes[1].legend(fontsize=14)

# Improve the layout
plt.tight_layout()

# Show the plot
plt.show()

### Evaluation of uncertainty

In [ ]:
# Assuming x(c) is an array of confidence levels from 0 to 1 at intervals of 0.01
confidence_levels = np.arange(0, 1.01, 0.01)

# Function to calculate the observed confidence
def calculate_observed_confidence(y_true, mean_pred, std_pred, z_value):
    lower_bound = mean_pred - z_value * std_pred / 2
    upper_bound = mean_pred + z_value * std_pred / 2
    return np.mean((y_true >= lower_bound) & (y_true <= upper_bound))

# Calculate the z-scores for the given confidence levels (two-tailed)
z_scores = [scipy.stats.norm.ppf((1 + cl) / 2) for cl in confidence_levels]

# Calculate the observed confidence for each z-score
observed_confidence = [calculate_observed_confidence(ytrain, mean_train, std_train, z) for z in z_scores]

# Plot the calibration curve
plt.figure(figsize=(6, 5))
plt.plot(confidence_levels, observed_confidence, label='Calibration curve')
plt.plot(confidence_levels, confidence_levels, 'k--', label='Perfect calibration')
plt.xlabel('Expected confidence')
plt.ylabel('Observed confidence')
plt.legend()
plt.show()

# Save the observed confidence data to a CSV file
BNN_calibration_data = pd.DataFrame({
    'Expected_Confidence': confidence_levels,
    'BNN_Observed_Confidence_Train': observed_confidence
})

In [ ]:
# Calculate the observed confidence for each z-score
observed_confidence_test = [calculate_observed_confidence(ytest, mean_test, std_test, z) for z in z_scores]

# Plot the calibration curve
plt.figure(figsize=(6, 5))
plt.plot(confidence_levels, observed_confidence_test, label='Calibration curve')
plt.plot(confidence_levels, confidence_levels, 'k--', label='Perfect calibration')
plt.xlabel('Expected confidence')
plt.ylabel('Observed confidence')
plt.legend()
plt.show()

# Save the data
BNN_calibration_data['BNN_Observed_Confidence_Test'] = observed_confidence

### Sparsification plots

In [ ]:
# Function to calculate RMSE using mean_squared_error from sklearn
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert them to NumPy arrays using .numpy() method

# Step 1: Sort samples by descending order of predictive uncertainty (standard deviation)
sorted_indices = np.argsort(-std_train)
sorted_ytrain = ytrain[sorted_indices]
sorted_mean_train = mean_train[sorted_indices]

# Step 2 and 3: Remove subsets of samples and calculate RMSE
rmse_values = []
fractions = np.arange(0.0, 1, 0.001)  # From 2% to 98% in steps of 2%

for fraction in fractions:
    # Calculate the number of samples to remove
    num_to_remove = int(fraction * len(sorted_ytrain))
    # Calculate RMSE on the remaining samples
    remaining_rmse = rmse(sorted_ytrain[num_to_remove:], sorted_mean_train[num_to_remove:])
    rmse_values.append(remaining_rmse)

# Step 4: Plot the error metric vs. fraction of removed samples
plt.figure(figsize=(6, 5))
plt.plot(fractions, rmse_values, marker='o')
plt.xlabel('Fraction of samples removed')
plt.ylabel('RMSE on remaining samples')
plt.title('Sparsification Plot')
plt.show()

# Define dataFrame
BNN_Sparsification_data = pd.DataFrame({
    'Sparsification_fractions': fractions,
    'BNN_rmse_values_Train': rmse_values
})

In [ ]:
# Step 1: Sort samples by descending order of predictive uncertainty (standard deviation)
sorted_indices_test = np.argsort(-std_test)
sorted_ytest = ytest[sorted_indices_test]
sorted_mean_test = mean_test[sorted_indices_test]

# Step 2 and 3: Remove subsets of samples and calculate RMSE
rmse_values_test = []

for fraction in fractions:
    # Calculate the number of samples to remove
    num_to_remove = int(fraction * len(sorted_ytest))
    # Calculate RMSE on the remaining samples
    remaining_rmse_test = rmse(sorted_ytest[num_to_remove:], sorted_mean_test[num_to_remove:])
    rmse_values_test.append(remaining_rmse_test)

# Step 4: Plot the error metric vs. fraction of removed samples
plt.figure(figsize=(6, 5))
plt.plot(fractions, rmse_values_test, marker='o')
plt.xlabel('Fraction of samples removed')
plt.ylabel('RMSE on remaining samples')
plt.title('Sparsification Plot')
plt.show()

# Save the data
BNN_Sparsification_data['BNN_rmse_values_Test'] = rmse_values_test

### Out of distribution data

In [ ]:
Corr_df = pickle.load(open("Corr_All.pickle","rb"))
unique_list = pickle.load(open("unique_list_All.pickle","rb"))
Columns = pickle.load(open("Columns_All.pickle","rb"))
Substructure_list = pickle.load(open("polymer.keys_All.pickle","rb"))

In [ ]:
data_OOD = pd.read_csv('../data/Tg_OOD_ME.csv')

molecules = data_OOD.Smiles.apply(Chem.MolFromSmiles)
fp = molecules.apply(lambda m: AllChem.GetMorganFingerprint(m, radius=3))
fp_n = fp.apply(lambda m: m.GetNonzeroElements())
MY_finger = []
for polymer in fp_n:
    my_finger = [0] * len(unique_list)
    for key in polymer.keys():
        if key in list(Corr_df[0]):
            index = Corr_df[Corr_df[0] == key]['index'].values[0]
            my_finger[index] = polymer[key]         
    MY_finger.append(my_finger)
X_OOD = pd.DataFrame(MY_finger)
X_OOD = X_OOD[Columns]

y_OOD = data_OOD['Tg'].values

# Converting to PyTorch tensors
x_OOD_tensor = torch.tensor(X_OOD.values).float()
y_OOD_tensor = torch.tensor(y_OOD).float()

# DataLoaders
OOD_dataset = TensorDataset(x_OOD_tensor, y_OOD_tensor)
OOD_loader = DataLoader(OOD_dataset, batch_size=bs)

In [ ]:
# Plot
kwargs = dict(hist_kws={'alpha':.3, 'edgecolor':'white'})
#plt.figure(figsize=(4,4), dpi= 600)
df_OOD = data_OOD
sns.histplot(df_OOD['Tg'].dropna(), kde=True, color = 'blue', alpha = 0.3, edgecolor='white')

#plt.xlim(-200,500)
#plt.legend()
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel("Tg [℃]",fontsize=18)
plt.ylabel("Count",fontsize=18) 

In [ ]:
mean_OOD, std_OOD = predict_with_uncertainty(model_BNN, x_OOD_tensor.to(device))

mean_OOD = mean_OOD.numpy()
std_OOD = std_OOD.numpy()

In [ ]:
# Calculate Spearman's Rank Correlation Coefficient for OOD data
abs_error_OOD = abs(y_OOD - mean_OOD)
spearman_corr_OOD, p_value_OOD = spearmanr(abs_error_OOD, std_OOD)

# Organize the results in a dictionary
spearman_results_OOD = {
    'Spearman_Correlation': [spearman_corr_OOD],
    'P_value': [p_value_OOD]
}

# Convert the dictionary to a DataFrame
spearman_df_OOD = pd.DataFrame(spearman_results_OOD, index=['OOD'])

# Display the DataFrame
spearman_df_OOD

In [ ]:
# Create the plot
plt.figure(figsize=(5, 5))
plt.scatter(abs_error_OOD, std_OOD, alpha=0.5)

# Add labels and title
plt.xlabel('Absolute Error (OOD)', fontsize=14)
plt.ylabel('Standard Deviation (OOD)', fontsize=14)
plt.title('Absolute Error vs Standard Deviation (OOD)', fontsize=16)

# Optionally, add grid for better readability
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Ensure that ytest, mean_predictions, and std_dev_predictions are 1D arrays
y_OOD_1d = np.ravel(y_OOD)
mean_OOD_1d = np.ravel(mean_OOD)
std_OOD_1d = np.ravel(std_OOD)

# Metric calculation
mae_OOD = mean_absolute_error(y_OOD_1d, mean_OOD_1d)
rmse_OOD = np.sqrt(mean_squared_error(y_OOD_1d, mean_OOD_1d))
r2_OOD = r2_score(y_OOD_1d, mean_OOD_1d)

# Organize the metrics into a dictionary with three keys for the three metrics
metrics_OOD = {
    'MAE': mae_OOD,
    'RMSE': rmse_OOD,
    'R2': r2_OOD
}

# Convert the dictionary to a DataFrame
metrics_OOD_df = pd.DataFrame(metrics_OOD, index=['OOD test'])

# Display the DataFrame
metrics_OOD_df

In [ ]:
# Create a figure for the OOD set plot
fig, ax = plt.subplots(figsize=(5.5, 5))

# Plotting for the test set
ax.errorbar(y_OOD_1d, mean_OOD_1d, 
            yerr=std_OOD_1d, 
            fmt='o', ecolor='lightblue', mec='green', mfc='lightgreen', 
            alpha=0.7, capsize=5, label='OOD data Prediction')

# Plot a line for perfect predictions for reference
ax.plot(y_OOD_1d, y_OOD_1d, 'r--', label='Perfect predictions')

# Define the ticks for the x and y axes
ax.set_xticks(np.arange(-100, 401, 100))
ax.set_yticks(np.arange(-100, 401, 100))

# Add labels and title
ax.set_xlabel('Actual Values', fontsize=14)
ax.set_ylabel('Predicted Values', fontsize=14)
ax.set_title('OOD data Predictions - Tg', fontsize=14)

# Add legend
ax.legend(fontsize=14)

# Improve the layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate the observed confidence for each z-score
observed_confidence = [calculate_observed_confidence(y_OOD, mean_OOD, 1*std_OOD, z) for z in z_scores]

# Plot the calibration curve
plt.figure(figsize=(6, 5))
plt.plot(confidence_levels, observed_confidence, label='Calibration curve')
plt.plot(confidence_levels, confidence_levels, 'k--', label='Perfect calibration')
plt.xlabel('Expected confidence')
plt.ylabel('Observed confidence')
plt.legend()
plt.show()

# Save the data
BNN_calibration_data['BNN_Observed_Confidence_OOD_EXP'] = observed_confidence

In [ ]:
# Function to calculate RMSE using mean_squared_error from sklearn

# Step 1: Sort samples by descending order of predictive uncertainty (standard deviation)
sorted_indices_OOD = np.argsort(-std_OOD)
sorted_y_OOD = y_OOD[sorted_indices_OOD]
sorted_mean_OOD = mean_OOD[sorted_indices_OOD]

# Step 2 and 3: Remove subsets of samples and calculate RMSE
rmse_values_OOD = []

for fraction in fractions:
    # Calculate the number of samples to remove
    num_to_remove = int(fraction * len(sorted_y_OOD))
    # Calculate RMSE on the remaining samples
    remaining_rmse_OOD = rmse(sorted_y_OOD[num_to_remove:], sorted_mean_OOD[num_to_remove:])
    rmse_values_OOD.append(remaining_rmse_OOD)

# Step 4: Plot the error metric vs. fraction of removed samples
plt.figure(figsize=(6, 5))
plt.plot(fractions, rmse_values_OOD, marker='o')
plt.xlabel('Fraction of samples removed')
plt.ylabel('RMSE on remaining samples')
plt.title('Sparsification Plot')
plt.show()

# Save the data
BNN_Sparsification_data['BNN_rmse_values_OOD_EXP'] = rmse_values_OOD

# OOD_MD data set

In [ ]:
data_OOD = pd.read_csv('../data/Tg_OOD_MD.csv')

molecules = data_OOD.Smiles.apply(Chem.MolFromSmiles)
fp = molecules.apply(lambda m: AllChem.GetMorganFingerprint(m, radius=3))
fp_n = fp.apply(lambda m: m.GetNonzeroElements())
MY_finger = []
for polymer in fp_n:
    my_finger = [0] * len(unique_list)
    for key in polymer.keys():
        if key in list(Corr_df[0]):
            index = Corr_df[Corr_df[0] == key]['index'].values[0]
            my_finger[index] = polymer[key]         
    MY_finger.append(my_finger)
X_OOD = pd.DataFrame(MY_finger)
X_OOD = X_OOD[Columns]

y_OOD = data_OOD['Tg'].values

# Converting to PyTorch tensors
x_OOD_tensor = torch.tensor(X_OOD.values).float()
y_OOD_tensor = torch.tensor(y_OOD).float()

# DataLoaders
OOD_dataset = TensorDataset(x_OOD_tensor, y_OOD_tensor)
OOD_loader = DataLoader(OOD_dataset, batch_size=bs)

In [ ]:
# Plot
kwargs = dict(hist_kws={'alpha':.3, 'edgecolor':'white'})
#plt.figure(figsize=(4,4), dpi= 600)
df_OOD_MD = data_OOD
sns.histplot(df_OOD_MD['Tg'].dropna(), kde=True, color = 'blue', alpha = 0.3, edgecolor='white')

#plt.xlim(-200,500)
#plt.legend()
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel("Tg [℃]",fontsize=18)
plt.ylabel("Count",fontsize=18)

In [ ]:
mean_OOD, std_OOD = predict_with_uncertainty(model_BNN, x_OOD_tensor.to(device))

mean_OOD = mean_OOD.numpy()
std_OOD = std_OOD.numpy()

In [ ]:
# Ensure that ytest, mean_predictions, and std_dev_predictions are 1D arrays
y_OOD_1d = np.ravel(y_OOD)
mean_OOD_1d = np.ravel(mean_OOD)
std_OOD_1d = np.ravel(std_OOD)

# Metric calculation
mae_OOD = mean_absolute_error(y_OOD_1d, mean_OOD_1d)
rmse_OOD = np.sqrt(mean_squared_error(y_OOD_1d, mean_OOD_1d))
r2_OOD = r2_score(y_OOD_1d, mean_OOD_1d)

# Organize the metrics into a dictionary with three keys for the three metrics
metrics_OOD = {
    'MAE': mae_OOD,
    'RMSE': rmse_OOD,
    'R2': r2_OOD
}

# Convert the dictionary to a DataFrame
metrics_OOD_df = pd.DataFrame(metrics_OOD, index=['OOD test'])

# Display the DataFrame
metrics_OOD_df

In [ ]:
# Create a figure for the OOD set plot
fig, ax = plt.subplots(figsize=(5.5, 5))

# Plotting for the test set
ax.errorbar(y_OOD_1d, mean_OOD_1d, 
            yerr=std_OOD_1d, 
            fmt='o', ecolor='lightblue', mec='green', mfc='lightgreen', 
            alpha=0.7, capsize=5, label='OOD data Prediction')

# Plot a line for perfect predictions for reference
ax.plot(y_OOD_1d, y_OOD_1d, 'r--', label='Perfect predictions')

# Define the ticks for the x and y axes
ax.set_xticks(np.arange(-100, 401, 100))
ax.set_yticks(np.arange(-100, 401, 100))

# Add labels and title
ax.set_xlabel('Actual Values', fontsize=14)
ax.set_ylabel('Predicted Values', fontsize=14)
ax.set_title('OOD data Predictions - Tg', fontsize=14)

# Add legend
ax.legend(fontsize=14)

# Improve the layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate Spearman's Rank Correlation Coefficient for OOD data
abs_error_OOD = abs(y_OOD - mean_OOD)
spearman_corr_OOD, p_value_OOD = spearmanr(abs_error_OOD, std_OOD)

# Organize the results in a dictionary
spearman_results_OOD = {
    'Spearman_Correlation': [spearman_corr_OOD],
    'P_value': [p_value_OOD]
}

# Convert the dictionary to a DataFrame
spearman_df_OOD = pd.DataFrame(spearman_results_OOD, index=['OOD'])

# Display the DataFrame
spearman_df_OOD

In [ ]:
# Create the plot
plt.figure(figsize=(5, 5))
plt.scatter(abs_error_OOD, std_OOD, alpha=0.5)

# Add labels and title
plt.xlabel('Absolute Error (OOD)', fontsize=14)
plt.ylabel('Standard Deviation (OOD)', fontsize=14)
plt.title('Absolute Error vs Standard Deviation (OOD)', fontsize=16)

# Optionally, add grid for better readability
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Calculate the observed confidence for each z-score
observed_confidence = [calculate_observed_confidence(y_OOD, mean_OOD, 1*std_OOD, z) for z in z_scores]

# Plot the calibration curve
plt.figure(figsize=(6, 5))
plt.plot(confidence_levels, observed_confidence, label='Calibration curve')
plt.plot(confidence_levels, confidence_levels, 'k--', label='Perfect calibration')
plt.xlabel('Expected confidence')
plt.ylabel('Observed confidence')
plt.legend()
plt.show()

# Save the data
BNN_calibration_data['BNN_Observed_Confidence_OOD_MD'] = observed_confidence

In [ ]:
# Function to calculate RMSE using mean_squared_error from sklearn
# Step 1: Sort samples by descending order of predictive uncertainty (standard deviation)
sorted_indices_OOD = np.argsort(-std_OOD)
sorted_y_OOD = y_OOD[sorted_indices_OOD]
sorted_mean_OOD = mean_OOD[sorted_indices_OOD]

# Step 2 and 3: Remove subsets of samples and calculate RMSE
rmse_values_OOD = []

for fraction in fractions:
    # Calculate the number of samples to remove
    num_to_remove = int(fraction * len(sorted_y_OOD))
    # Calculate RMSE on the remaining samples
    remaining_rmse_OOD = rmse(sorted_y_OOD[num_to_remove:], sorted_mean_OOD[num_to_remove:])
    rmse_values_OOD.append(remaining_rmse_OOD)

# Step 4: Plot the error metric vs. fraction of removed samples
plt.figure(figsize=(7, 6))
plt.plot(fractions, rmse_values_OOD, marker='o')
plt.xlabel('Fraction of samples removed')
plt.ylabel('RMSE on remaining samples')
plt.title('Sparsification Plot')
plt.show()

# Save the data
BNN_Sparsification_data['BNN_rmse_values_OOD_MD'] = rmse_values_OOD

### data saving

In [ ]:
# Define the filename
filename = '../results/BNN_calibration_data.csv'
# Save to CSV
BNN_calibration_data.to_csv(filename, index=False)
BNN_calibration_data 

In [ ]:
# Define the filename
filename2 = '../results/BNN_Sparsification_data.csv'
# Save to CSV
BNN_Sparsification_data.to_csv(filename2, index=False)
BNN_Sparsification_data 

### high Tg

In [ ]:
data_OOD = pd.read_csv('../data/high_Tg.csv')

molecules = data_OOD.Smiles.apply(Chem.MolFromSmiles)
fp = molecules.apply(lambda m: AllChem.GetMorganFingerprint(m, radius=3))
fp_n = fp.apply(lambda m: m.GetNonzeroElements())
MY_finger = []
for polymer in fp_n:
    my_finger = [0] * len(unique_list)
    for key in polymer.keys():
        if key in list(Corr_df[0]):
            index = Corr_df[Corr_df[0] == key]['index'].values[0]
            my_finger[index] = polymer[key]         
    MY_finger.append(my_finger)
X_OOD = pd.DataFrame(MY_finger)
X_OOD = X_OOD[Columns]

y_OOD = data_OOD['Tg'].values

# Converting to PyTorch tensors
x_OOD_tensor = torch.tensor(X_OOD.values).float()
y_OOD_tensor = torch.tensor(y_OOD).float()

# DataLoaders
OOD_dataset = TensorDataset(x_OOD_tensor, y_OOD_tensor)
OOD_loader = DataLoader(OOD_dataset, batch_size=bs)

In [ ]:
# Plot
kwargs = dict(hist_kws={'alpha':.3, 'edgecolor':'white'})
#plt.figure(figsize=(4,4), dpi= 600)
df_OOD_MD = data_OOD
sns.histplot(df_OOD_MD['Tg'].dropna(), kde=True, color = 'blue', alpha = 0.3, edgecolor='white')

#plt.xlim(-200,500)
#plt.legend()
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel("Tg [℃]",fontsize=18)
plt.ylabel("Count",fontsize=18)

In [ ]:
mean_OOD, std_OOD = predict_with_uncertainty(model_BNN, x_OOD_tensor.to(device))

mean_OOD = mean_OOD.numpy()
std_OOD = std_OOD.numpy()

In [ ]:
ci_multiplier = 1.96  # Multiplier for a 95% confidence interval in a normal distribution
lower_bound = mean_OOD - ci_multiplier * std_OOD
upper_bound = mean_OOD + ci_multiplier * std_OOD

# Create a DataFrame with the results
df = pd.DataFrame({
    'mean_OOD': mean_OOD,
    'std_OOD': std_OOD,
    '95% CI Lower': lower_bound,
    '95% CI Upper': upper_bound
})

# Output the DataFrame
print(df)

excel_file_path = '../results/high_Tg_BNN.csv'  # Path where the Excel file will be saved
df.to_csv(excel_file_path, index=False)


In [ ]:
# Ensure that ytest, mean_predictions, and std_dev_predictions are 1D arrays
y_OOD_1d = np.ravel(y_OOD)
mean_OOD_1d = np.ravel(mean_OOD)
std_OOD_1d = np.ravel(std_OOD)

# Metric calculation
mae_OOD = mean_absolute_error(y_OOD_1d, mean_OOD_1d)
rmse_OOD = np.sqrt(mean_squared_error(y_OOD_1d, mean_OOD_1d))
r2_OOD = r2_score(y_OOD_1d, mean_OOD_1d)

# Organize the metrics into a dictionary with three keys for the three metrics
metrics_OOD = {
    'MAE': mae_OOD,
    'RMSE': rmse_OOD,
    'R2': r2_OOD
}

# Convert the dictionary to a DataFrame
metrics_OOD_df = pd.DataFrame(metrics_OOD, index=['OOD test'])

# Display the DataFrame
metrics_OOD_df

In [ ]:
font_size = 16
# Create a figure for the OOD set plot
# plt.figure(figsize=(7, 6), dpi=1200)
fig, ax = plt.subplots(figsize=(5, 4.5), dpi=1200)
# Plotting for the test set
ax.errorbar(y_OOD_1d, mean_OOD_1d, 
            yerr=std_OOD_1d, 
            fmt='o', ecolor='lightblue', mec='green', mfc='lightgreen', 
            alpha=0.7, capsize=5, label='High Tg Prediction')

# Plot a line for perfect predictions for reference
ax.plot((250, 520), (250, 520), 'r--', label='Perfect predictions')

# Define the ticks for the x and y axes
ax.set_xticks(np.arange(250, 520, 40))
ax.set_yticks(np.arange(250, 520, 40))

# Add labels and title
ax.set_xlabel('Actual Values', fontsize=16, weight='bold')
ax.set_ylabel('Predicted Values', fontsize=16, weight='bold')
# Fixing the fontsize setting for ticks
ax.tick_params(axis='both', which='major', labelsize=16)
plt.rc('font', weight='bold')
plt.rc('axes', linewidth=2)
# Add legend
ax.legend(fontsize=font_size, frameon=False)


# Improve the layout
plt.tight_layout()
plt.savefig('../results/figure_high_Tg/BNN.png', format='png', bbox_inches='tight')
# Show the plot
plt.show()